In [1]:
import numpy as np
import pandas as pd
import random
import time
import math
import os
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn import init
from dateutil import parser
from pathlib import Path
import json 
import shutil
import logging
logger = logging.getLogger(str(os.getpid()))

from wattile.data_reading import read_dataset_from_file
from wattile.buildings_processing import correct_predictor_columns, correct_timestamps, resample_or_rolling_stats, timelag_predictors, timelag_predictors_target, roll_predictors_target
from wattile.time_processing import add_processed_time_columns
PROJECT_DIRECTORY = Path().resolve().parent.parent

# reading configs

In [2]:
"""
For this example, we will be using the default configs.
Check out the docs for an explaination of each config.
"""
##################################################################################
# choose the configs file to use as an input
##################################################################################
# main configs file
with open(PROJECT_DIRECTORY / "wattile" / "configs" / "configs.json", "r") as f:
    configs = json.load(f)
##################################################################################
# code testing configs file
# with open(PROJECT_DIRECTORY / "tests" / "fixtures" / "test_configs.json", "r") as f:
#     configs = json.load(f)
##################################################################################

exp_dir = PROJECT_DIRECTORY / "notebooks" / "exp_dir"
if exp_dir.exists():
    shutil.rmtree(exp_dir)
exp_dir.mkdir()

configs["data_input"]["exp_dir"] = str(PROJECT_DIRECTORY / exp_dir)
configs["data_input"]["data_dir"] = str(PROJECT_DIRECTORY / "data" / "Synthetic Site")

configs

{'data_input': {'data_dir': 'C:\\Users\\JKIM4\\Documents\\GitHub\\intelligentcampus-pred-analytics\\data\\Synthetic Site',
  'data_config': 'Synthetic Site Config.json',
  'start_time': '2018-01-01T00:00:00-07:00',
  'end_time': '2022-01-01T00:00:00-07:00',
  'predictor_columns': ['Synthetic Weather Station Dew Point Temperature',
   'Synthetic Weather Station Diffuse Horizontal Irradiance',
   'Synthetic Weather Station Direct Normal Irradiance',
   'Synthetic Weather Station Dry Bulb Temperature',
   'Synthetic Weather Station Global Horizontal Irradiance',
   'Synthetic Weather Station Relative Humidity',
   'Synthetic Weather Station Wind Speed'],
  'target_var': 'Synthetic Site Electricity Main Total Power',
  'exp_dir': 'C:\\Users\\JKIM4\\Documents\\GitHub\\intelligentcampus-pred-analytics\\notebooks\\exp_dir'},
 'data_output': {'exp_dir': 'exp_dir',
  'plot_comparison': True,
  'plot_comparison_portion_start': 0.0,
  'plot_comparison_portion_end': 1.0},
 'data_processing': {'fea

# reading data

In [3]:
configs["target_feat_name"] = [configs["data_input"]["target_var"]]
data = read_dataset_from_file(configs)
data

,Synthetic Weather Station Dew Point Temperature,Synthetic Weather Station Diffuse Horizontal Irradiance,Synthetic Weather Station Direct Normal Irradiance,Synthetic Weather Station Dry Bulb Temperature,Synthetic Weather Station Global Horizontal Irradiance,Synthetic Weather Station Relative Humidity,Synthetic Weather Station Wind Speed,Synthetic Site Electricity Main Total Power
Timestamp,,,,,,,,
2021-12-01 07:00:00+00:00,15.413733,-1.525850,-0.005199,56.408001,-1.810825,17.930000,10.457981,104.800003
2021-12-01 07:01:00+00:00,15.183906,-1.496226,0.207964,56.174000,-1.841866,17.879999,11.265539,96.650002
2021-12-01 07:02:00+00:00,14.891221,-1.441191,0.457520,55.886002,-1.873579,17.809999,12.777752,96.199997
2021-12-01 07:03:00+00:00,14.836164,-1.371754,0.473117,55.796001,-1.860084,17.820000,12.694983,95.599998
2021-12-01 07:04:00+00:00,14.782966,-1.317349,0.545903,55.723999,-1.843271,17.820000,11.632407,100.650002
...,...,...,...,...,...,...,...,...
2021-12-08 06:55:00+00:00,11.210565,-1.487212,-0.254757,42.285198,-1.814939,24.680000,0.000000,109.664803
2021-12-08 06:56:00+00:00,11.190062,-1.445182,-0.233960,41.997200,-1.852901,24.930000,0.000000,107.002800
2021-12-08 06:57:00+00:00,11.223961,-1.396302,-0.145575,41.669601,-1.841248,25.290001,3.545647,106.480400


# data processing 1

In [4]:
# Add time-based features
data = add_processed_time_columns(data, configs)

data

,Synthetic Weather Station Dew Point Temperature,Synthetic Weather Station Diffuse Horizontal Irradiance,Synthetic Weather Station Direct Normal Irradiance,Synthetic Weather Station Dry Bulb Temperature,Synthetic Weather Station Global Horizontal Irradiance,Synthetic Weather Station Relative Humidity,Synthetic Weather Station Wind Speed,Synthetic Site Electricity Main Total Power,sin_HOD,cos_HOD,...,DOW_binary_reg_6,DOW_binary_fuzzy_0,DOW_binary_fuzzy_1,DOW_binary_fuzzy_2,DOW_binary_fuzzy_3,DOW_binary_fuzzy_4,DOW_binary_fuzzy_5,DOW_binary_fuzzy_6,sin_MOY,cos_MOY
Timestamp,,,,,,,,,,,,,,,,,,,,,
2021-12-01 07:00:00+00:00,15.413733,-1.525850,-0.005199,56.408001,-1.810825,17.930000,10.457981,104.800003,0.965926,-0.258819,...,0,0.0,0.0,0.708333,0.291667,0.0,0.0,0.0,-0.493776,0.869589
2021-12-01 07:01:00+00:00,15.183906,-1.496226,0.207964,56.174000,-1.841866,17.879999,11.265539,96.650002,0.964787,-0.263031,...,0,0.0,0.0,0.708333,0.291667,0.0,0.0,0.0,-0.493776,0.869589
2021-12-01 07:02:00+00:00,14.891221,-1.441191,0.457520,55.886002,-1.873579,17.809999,12.777752,96.199997,0.963630,-0.267238,...,0,0.0,0.0,0.708333,0.291667,0.0,0.0,0.0,-0.493776,0.869589
2021-12-01 07:03:00+00:00,14.836164,-1.371754,0.473117,55.796001,-1.860084,17.820000,12.694983,95.599998,0.962455,-0.271440,...,0,0.0,0.0,0.708333,0.291667,0.0,0.0,0.0,-0.493776,0.869589
2021-12-01 07:04:00+00:00,14.782966,-1.317349,0.545903,55.723999,-1.843271,17.820000,11.632407,100.650002,0.961262,-0.275637,...,0,0.0,0.0,0.708333,0.291667,0.0,0.0,0.0,-0.493776,0.869589
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-08 06:55:00+00:00,11.210565,-1.487212,-0.254757,42.285198,-1.814939,24.680000,0.000000,109.664803,0.971342,-0.237686,...,0,0.0,0.0,0.750000,0.250000,0.0,0.0,0.0,-0.385663,0.922640
2021-12-08 06:56:00+00:00,11.190062,-1.445182,-0.233960,41.997200,-1.852901,24.930000,0.000000,107.002800,0.970296,-0.241922,...,0,0.0,0.0,0.750000,0.250000,0.0,0.0,0.0,-0.385663,0.922640
2021-12-08 06:57:00+00:00,11.223961,-1.396302,-0.145575,41.669601,-1.841248,25.290001,3.545647,106.480400,0.969231,-0.246153,...,0,0.0,0.0,0.750000,0.250000,0.0,0.0,0.0,-0.385663,0.922640


In [6]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.colors import n_colors
from plotly.validators.scatter.marker import SymbolValidator
import plotly.io as pio
os.environ['path'] += r';C:/Users/JKIM4/Downloads/vips-dev-w64-all-8.11.0/vips-dev-8.11/bin'
import pyvips

In [22]:
path = "../../tests/fixtures"

In [25]:
colorscale = "Earth"

list_line = ["solid", "dot", "dash", "longdash", "dashdot", "longdashdot"]

n_colors = len(list_line)
list_colors = px.colors.sample_colorscale(colorscale, [n/(n_colors -1) for n in range(n_colors)])
random.shuffle(list_colors)

df_feat_time = data.loc[:, data.columns.str.contains("MOY|DOW|HOD")]
df_feat_time

dict_daterange = {
    "HOD":["2021-12-02 00:00:00+00:00","2021-12-03 00:00:00+00:00"],
    "DOW":["2021-12-02 00:00:00+00:00","2021-12-08 00:00:00+00:00"],
    "MOY":["2021-12-01 07:00:00+00:00","2021-12-08 06:59:00+00:00"]
}

dict_feat_include = {
    "HOD":[
        "sin_HOD",
        "cos_HOD",
        "HOD_binary_reg_10",
        "HOD_binary_reg_20",
        "HOD_binary_fuzzy_13",
        "HOD_binary_fuzzy_23"
    ],
    "DOW":[
        "DOW_binary_reg_0",
        "DOW_binary_reg_2",
        "DOW_binary_reg_4",
        "DOW_binary_fuzzy_1",
        "DOW_binary_fuzzy_3",
        "DOW_binary_fuzzy_5",
    ],
    "MOY":[
        "sin_MOY",
        "cos_MOY",
    ],
}

i=0
for feat in dict_daterange.keys():
    fig = go.Figure()
    
    df_temp = data.loc[dict_daterange[feat][0]:dict_daterange[feat][1], data.columns.str.contains(feat)][dict_feat_include[feat]]
    
    if feat == "DOW":
        df_temp = df_temp.resample("60min").mean()
    
    i_label = 0
    for col in df_temp.columns:
        
        fig.add_trace(go.Scatter(
            mode="lines",
            x=df_temp.index.values,
            y=df_temp[col].values,
            name=col,
            line=dict(
                color=list_colors[i_label],
                dash=list_line[i_label],
            ),
        ))
        
        i_label+=1

    fig.update_layout(
        width=800,
        height=200,
        margin=dict(
            l=0,
            r=0,
            t=0,
            b=0,
        ),
        plot_bgcolor="rgb(245,245,245)",
    )

    filename = "example_feat_time_{}".format(feat)
    pio.write_image(fig, path + "/" + filename + ".svg")
    # https://stackoverflow.com/questions/51450134/how-to-convert-svg-to-png-or-jpeg-in-python
    image = pyvips.Image.thumbnail(path + "/" + filename + ".svg", 3000)
    image.write_to_file(path + "/" + filename + ".png")

    fig.show()
    
#     if i== 1:
#         break
        
    i+=1

,DOW_binary_reg_0,DOW_binary_reg_2,DOW_binary_reg_4,DOW_binary_fuzzy_1,DOW_binary_fuzzy_3,DOW_binary_fuzzy_5
Timestamp,,,,,,
2021-12-02 00:00:00+00:00,0.0,0.0,0.0,0.000000,1.000000,0.0
2021-12-02 01:00:00+00:00,0.0,0.0,0.0,0.000000,0.958333,0.0
2021-12-02 02:00:00+00:00,0.0,0.0,0.0,0.000000,0.916667,0.0
2021-12-02 03:00:00+00:00,0.0,0.0,0.0,0.000000,0.875000,0.0
2021-12-02 04:00:00+00:00,0.0,0.0,0.0,0.000000,0.833333,0.0
...,...,...,...,...,...,...
2021-12-07 20:00:00+00:00,0.0,0.0,0.0,0.166667,0.000000,0.0
2021-12-07 21:00:00+00:00,0.0,0.0,0.0,0.125000,0.000000,0.0
2021-12-07 22:00:00+00:00,0.0,0.0,0.0,0.083333,0.000000,0.0


In [ ]:
# Add statistics features
data = resample_or_rolling_stats(data, configs)